## 1. Introduction to Modeling and Simulation
- **Definitions**:
  - **Modeling**: The process of creating a representation of a system or process. This model can be a physical, mathematical, or logical representation. It helps in understanding, analyzing, and predicting the behavior of systems.
    
  - **Simulation**: The use of models to conduct experiments and study the behavior of the system under different conditions. Simulation helps in testing hypotheses, visualizing complex processes, and making informed decisions.


- **Applications in Digital Engineering**:
  - **Design Optimization**: Using simulations to explore various design alternatives and identify the most efficient, cost-effective, and high-performance solutions.
    
  - **Manufacturing Process Improvements**: Implementing simulations to refine manufacturing processes, reducing waste, enhancing productivity, and ensuring quality control.


## 2. Key Concepts of Modeling and Simulation

- **Traditional Engineering Analysis (FEA):**
  Finite Element Analysis (FEA) is a computational method used to predict how physical phenomena like stress, heat, and vibration affect materials and structures. Engineers use FEA to create models that simulate these effects and help them understand potential weaknesses or points of failure. The focus is on interpreting the outputs of FEA, such as stress/strain plots and heat maps, rather than on running the simulations themselves.

  <center> <img src="Module 4/FEA Example.png" alt="Description" width="600"/> </center>

- **Data-Driven Models:**
  Data-driven models leverage machine learning and statistical analysis to enhance decision-making and optimization in manufacturing processes. These models analyze historical data to predict future outcomes, such as equipment failures or quality control issues. By identifying patterns and trends, data-driven models enable proactive maintenance and continuous improvement, leading to increased efficiency and reduced downtime.

<font color='#0024CC'>

**Example**: This is an example of a **data-driven model**. The interactive widget allows you to increase the model's complexity (degree of the polynomial) to achieve a better fit to the data.

In this case, the model is purely data-driven, relying on regression techniques to find an equation that best describes the relationship between the variables. The model **does not incorporate any prior knowledge about the underlying physics** governing the system. Instead, it relies entirely on the data points provided.

The equation produced by the model is of the form:

$$
y = a_0 + a_1x + a_2x^2 + \dots + a_nx^n
$$

Where:

- \( y \) is the output (e.g., power consumption),
- \( x \) is the input (e.g., temperature),
- \( a_0, a_1, \dots, a_n \) are the coefficients determined by the regression process,
- \( n \) is the degree of the polynomial (adjustable in the widget).

</font>


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import ipywidgets as widgets
from IPython.display import display

# Generate a more fluctuating dataset with conditional variability
np.random.seed(42)
temperature = np.linspace(20, 50, 30)

# Add conditional variability to the noise
noise = np.where(temperature < 35,  # Condition for temperature < 35
                 np.random.uniform(0, 50, size=temperature.shape),  # Low variability
                 np.random.uniform(0, 150, size=temperature.shape))  # High variability

# Calculate power consumption with conditional noise
power_consumption = 100 + 3 * temperature + 0.1 * temperature**2 + noise


# Reshape the data for the model
temperature = temperature.reshape(-1, 1)

# Create an output widget for the plot
output = widgets.Output()

def update_plot(degree):
    with output:
        output.clear_output(wait=True)  # Clear only the plot output
        # Generate polynomial features
        poly = PolynomialFeatures(degree=degree)
        temperature_poly = poly.fit_transform(temperature)
        
        # Create and fit the polynomial regression model
        model = LinearRegression()
        model.fit(temperature_poly, power_consumption)
        
        # Get the coefficients of the model
        coefficients = model.coef_
        intercept = model.intercept_
        
        # Create the model equation
        terms = [f"{coeff:.2f} * x^{i}" for i, coeff in enumerate(coefficients) if coeff != 0]
        equation = " + ".join(terms) + f" + {intercept:.2f}"
        
        # Predict power consumption for the given temperature data
        predicted_power_consumption = model.predict(temperature_poly)
        
        # Plot the data and the fitted line
        plt.figure(figsize=(6, 4))
        plt.scatter(temperature, power_consumption, color='blue', label='Data')
        plt.plot(temperature, predicted_power_consumption, color='red', label='Model')
        plt.xlabel('Temperature (°C)')
        plt.ylabel('Power Consumption (kW)')
        # plt.title(f'Temperature vs. Power Consumption (Degree {degree} Polynomial)')
        plt.legend()
        plt.grid()
        plt.ylim([150,700])
        plt.show()
        
        # Display the model equation
        print(f"Fitted Equation: {equation}")

# Create a slider for varying the degree of the polynomial
degree_slider = widgets.IntSlider(min=1, max=10, step=1, value=1, description='Degree')

# Link the slider to the update_plot function
widgets.interactive_output(update_plot, {'degree': degree_slider})

# Display the widget and the plot output
display(degree_slider, output)

# Manually call the function once to display the initial plot
update_plot(degree_slider.value)


IntSlider(value=1, description='Degree', max=10, min=1)

Output()

- **Physics-Based Data Models:**
  Physics-based data models combine the principles of physical laws with real-world data to create simulations that offer deeper insights into complex systems. These models use equations derived from physics to simulate the behavior of systems under various conditions. By integrating data into these simulations, engineers can achieve a more accurate and comprehensive understanding of how systems perform in real-world scenarios.

<font color='#0024CC'>
    

**Example**: In contrast to the purely data-driven approach, the **physics-based data-driven model** combines domain knowledge (physical laws) with data-driven methods. Here, we start with an idea of the equation that describes the system, based on physical principles. For this problem, we use the following physics-based equation:

$$
y_{\text{physics}} = 100 + 3x + 0.1x^2
$$

Where:
- **y_{physics}** is the power consumption predicted from the physics model,
- **x** is the temperature.

However, real-world systems often exhibit complexities (e.g., noise, nonlinearities) that are not fully captured by the physics-based equation. To address this, we introduce a **data-driven residual correction**, which models the difference between the physics-based prediction and the observed data:

$$
y = y_{\text{physics}} + y_{\text{residual}}
$$

Where:
- **y_{physics}** is the prediction from the physics-based equation,
- **y_{residual}** is the residual, modeled using regression techniques (data-driven).

**Benefits of the Physics-Based Approach**:
1. **Interpretability**: The physics-based component provides insights into the underlying mechanisms of the system.
2. **Generalization**: By grounding the model in physical principles, it is more robust to extrapolation beyond the data range.
3. **Efficiency**: The physics-based component reduces the complexity of the data-driven part, requiring fewer parameters to achieve a good fit.

</font>


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import ipywidgets as widgets
from IPython.display import display

# Generate a more fluctuating dataset with conditional variability
np.random.seed(42)
temperature = np.linspace(20, 50, 30)

# Add conditional variability to the noise
noise = np.where(temperature < 35,  # Condition for temperature < 35
                 np.random.uniform(0, 50, size=temperature.shape),  # Low variability
                 np.random.uniform(0, 150, size=temperature.shape))  # High variability

# Physics-based model for power consumption (e.g., quadratic relationship with temperature)
def physics_model(temp):
    return 100 + 3 * temp + 0.1 * temp**2

# Calculate power consumption with physics-based model and noise
power_consumption = physics_model(temperature) + noise

# Reshape the data for the machine learning model
temperature = temperature.reshape(-1, 1)

# Create an output widget for the plot
output = widgets.Output()

def update_plot(degree):
    with output:
        output.clear_output(wait=True)  # Clear only the plot output
        # Generate polynomial features
        poly = PolynomialFeatures(degree=degree)
        temperature_poly = poly.fit_transform(temperature)
        
        # Fit the physics-based residuals using the machine learning model
        physics_predictions = physics_model(temperature.flatten())
        residuals = power_consumption - physics_predictions
        
        model = LinearRegression()
        model.fit(temperature_poly, residuals)
        
        # Predict residuals and combine with physics-based predictions
        predicted_residuals = model.predict(temperature_poly)
        predicted_power_consumption = physics_predictions + predicted_residuals
        
        # Get the coefficients of the residual model
        coefficients = model.coef_
        intercept = model.intercept_
        
        # Create the model equation
        terms = [f"{coeff:.2f} * x^{i}" for i, coeff in enumerate(coefficients) if coeff != 0]
        equation = " + ".join(terms) + f" + {intercept:.2f}"
        
        # Plot the data and the fitted line
        plt.figure(figsize=(6, 4))
        plt.scatter(temperature, power_consumption, color='blue', label='Data')
        plt.plot(temperature, physics_predictions, color='green', linestyle='--', label='Physics Model')
        plt.plot(temperature, predicted_power_consumption, color='red', label='Physics + Data Model')
        
        plt.xlabel('Temperature (°C)')
        plt.ylabel('Power Consumption (kW)')
        plt.grid()
        plt.ylim([150, 700])
        plt.legend()
        plt.show()
        
        # Display the model equation
        print(f"Fitted Residuals Equation: {equation}")

# Create a slider for varying the degree of the polynomial
degree_slider = widgets.IntSlider(min=1, max=10, step=1, value=1, description='Degree')

# Link the slider to the update_plot function
widgets.interactive_output(update_plot, {'degree': degree_slider})

# Display the widget and the plot output
display(degree_slider, output)

# Manually call the function once to display the initial plot
update_plot(degree_slider.value)


IntSlider(value=1, description='Degree', max=10, min=1)

Output()

### 3. Optimization Concepts Using Simulation Outputs
- **What is optimization?**
  Optimization is the process of finding the best possible design or performance by systematically adjusting variables to achieve desired outcomes. It involves using mathematical and computational techniques to maximize or minimize certain objectives while adhering to specific constraints.

  - **Using simulation outputs to find the best design/performance**:
    Simulations provide valuable data on the behavior and performance of systems under various conditions. By analyzing these outputs, engineers can identify optimal design parameters that meet performance criteria and constraints. This iterative process involves running multiple simulations, adjusting variables, and evaluating results to converge on the best solution.

  - **Applications in reducing material usage, improving strength, and minimizing defects**:
    1. **Reducing material usage**: Optimization techniques can help identify ways to use materials more efficiently, reducing waste and cost. For example, lightweighting strategies in automotive and aerospace industries optimize material distribution to maintain strength while minimizing weight.
    
    2. **Improving strength**: By optimizing design parameters, engineers can enhance the structural integrity and strength of components. This includes identifying the optimal geometry, material properties, and manufacturing processes to achieve the desired performance.
    
    3. **Minimizing defects**: Optimization can also focus on improving manufacturing processes to reduce defects and increase quality. This involves analyzing simulation data to identify process parameters that minimize variations and defects, leading to higher consistency and reliability in production.